In [ ]:
import pandas as pd
import pyodbc
import pyarrow as pa
import pyarrow.parquet as pq
import os
import shutil
import glob
from datetime import date, datetime
from time import time

In [ ]:
# Variáveis de data e hora
ref_data = date.today()
ref_year = date.today().year
ref_time = datetime.now().strftime("%H%M%S")

In [ ]:
# Parametros para a criação da string de conexão ao banco sql
server = 'localhost'
database = 'db_datasus'
username = 'sa'
password = 'Ribeiro83'

# String de conexão ao banco sql
conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+password)

query = """SELECT [date] 
                ,[state] 
                ,SUM([confirmed]) AS [confirmed] 
                ,SUM([deaths]) AS [deaths] 
                ,[estimated_population_2019] 
                ,[estimated_population] 
                ,[city_ibge_code] 
                ,[confirmed_per_100k_inhabitants] 
                ,[death_rate] 
            FROM [db_datasus].[dbo].[COVID] 
            WHERE city_ibge_code < 100 
            GROUP BY  
                  [date] 
                  ,[state] 
                  ,[estimated_population_2019] 
                  ,[estimated_population] 
                  ,[city_ibge_code] 
                  ,[confirmed_per_100k_inhabitants] 
                  ,[death_rate] """

sql_query = pd.read_sql_query(query, conn)

In [ ]:
# Cria o dataset pandas a partir da consulta sql
df = pd.DataFrame(sql_query)

In [ ]:
# df.to_parquet('df.parquet.snappy',compression='snappy')
# pd.read_parquet('df.parquet.snappy')

In [ ]:
# Realiza a conversa do dataset em pandas para uma table
table = pa.Table.from_pandas(df)

In [ ]:
# Declara as variáveis de dia e hora corrente para montar o nome do arquivo
date = datetime.now().strftime('%Y%m%d')
time = str(ref_time)

# Pasta e nome do arquivo no diretório temporário
filename = 'C:/Temp/stage/etl_hdfs'+'_'+date +'_'+ time

# Convert a table para o arquivo .parquet
pq.write_table(table, filename+'.parquet')

In [ ]:
# Diretorio do ano corrente que será criado
dir_ano = str(ref_year)
  
# Caminho do diretório pai
dir_pai = "C:/Users/tiago/Desktop/"
  
# Caminho a ser criado
path = os.path.join(dir_pai, dir_ano)
  
# Criar o diretorio 'ANO' em 'C:/Users/tiago/Desktop/'
if not os.path.exists(path):
    os.makedirs(path)

In [ ]:
# Diretorio do mês corrente que será criado
dir_mes = datetime.now().strftime('%m')
  
# Caminho do diretório pai
dir_pai = "C:/Users/tiago/Desktop/"+dir_ano
  
# Caminho a ser criado
path = os.path.join(dir_pai, dir_mes)
  
# Criar o diretorio 'MÊS' em 'C:/Users/tiago/Desktop/'
if not os.path.exists(path):
    os.makedirs(path)

In [ ]:
# Diretorio do dia corrente que será criado
dir_dia = datetime.now().strftime('%d')
  
# Caminho do diretório pai
dir_pai = "C:/Users/tiago/Desktop/"+dir_ano+'/'+dir_mes

# Caminho a ser criado
path = os.path.join(dir_pai, dir_dia)
  
# Criar o diretorio 'DIA' em 'C:/Users/tiago/Desktop/'
if not os.path.exists(path):
    os.makedirs(path)

In [ ]:
# Caminho de origem do arquivo .parquet
source_dir = 'C:/Temp/stage'
sources = glob.glob(os.path.join(source_dir,"*.parquet"))

# Caminho de destino do arquivo .parquet
destination = 'C:/Users/tiago/Desktop/'+str(dir_ano)+'/'+str(dir_mes)+'/'+str(dir_dia)
# Move o arquivo .parquet do caminho de origem para o de destino
for source in sources:
    if os.path.isfile(source):
        shutil.move(source,destination)